Analysis of missed slots on Ethereum mainnet.

A **missed slot** is a slot where the assigned proposer failed to produce a block. The chain skips these slots and continues. Typical miss rate is ~0.3%.

**Definition:** `slot exists in proposer_duty AND no block exists in canonical_beacon_block`

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML, display

from loaders import load_parquet, display_sql

target_date = None  # Set via papermill, or auto-detect from manifest

In [ ]:
display_sql("block_production_timeline", target_date)

In [ ]:
df = pl.from_pandas(load_parquet("block_production_timeline", target_date))

# Identify missed slots: ClickHouse LEFT JOIN returns epoch date (1970-01-01) instead of NULL
# for non-matching rows, so we detect missed slots by checking for the epoch timestamp
epoch = pl.datetime(1970, 1, 1)
df = df.with_columns(
    (pl.col("block_first_seen") == epoch).alias("is_missed")
)

total_slots = len(df)
missed_slots = df["is_missed"].sum()
produced_slots = total_slots - missed_slots
miss_rate = missed_slots / total_slots * 100

print(f"Total slots: {total_slots:,}")
print(f"Blocks produced: {produced_slots:,} ({produced_slots/total_slots*100:.2f}%)")
print(f"Missed slots: {missed_slots:,} ({miss_rate:.2f}%)")

## Missed slots by proposer entity

Which staking entities had the most missed slots? Note that larger entities have more assigned slots, so absolute counts don't reflect performance.

In [ ]:
# Missed slots by entity
df_missed = df.filter(pl.col("is_missed"))

if len(df_missed) > 0:
    # Fill empty entities
    df_missed = df_missed.with_columns(
        pl.col("proposer_entity").fill_null("unknown").replace("", "unknown").alias("proposer_entity")
    )
    
    entity_misses = (
        df_missed.group_by("proposer_entity")
        .agg(pl.len().alias("missed_count"))
        .sort("missed_count")
    )
    
    # Convert to pandas for plotting
    entity_misses_pd = entity_misses.to_pandas()
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=entity_misses_pd["proposer_entity"],
        x=entity_misses_pd["missed_count"],
        orientation="h",
        marker_color="#e74c3c",
        text=entity_misses_pd["missed_count"],
        textposition="outside",
    ))
    fig.update_layout(
        margin=dict(l=150, r=50, t=30, b=60),
        xaxis=dict(title="Missed slots"),
        yaxis=dict(title=""),
        height=max(300, len(entity_misses_pd) * 25 + 100),
    )
    fig.show(config={"responsive": True})
else:
    print("No missed slots today.")

## Entity miss rates

Normalized view: what percentage of each entity's assigned slots were missed? This accounts for entity size.

In [ ]:
if len(df_missed) > 0:
    # Calculate miss rate per entity
    df_with_entity = df.with_columns(
        pl.col("proposer_entity").fill_null("unknown").replace("", "unknown").alias("proposer_entity_clean")
    )
    
    entity_stats = (
        df_with_entity.group_by("proposer_entity_clean")
        .agg(
            pl.len().alias("total_slots"),
            pl.col("is_missed").sum().alias("missed_slots")
        )
        .with_columns(
            (pl.col("missed_slots") / pl.col("total_slots") * 100).alias("miss_rate")
        )
    )
    
    # Only show entities with at least 1 missed slot
    entity_stats = (
        entity_stats.filter(pl.col("missed_slots") > 0)
        .sort("miss_rate")
    )
    
    # Convert to pandas for plotting
    entity_stats_pd = entity_stats.to_pandas()
    
    # Create text labels
    entity_stats_pd["label"] = entity_stats_pd.apply(
        lambda r: f"{r['miss_rate']:.1f}% ({int(r['missed_slots'])}/{int(r['total_slots'])})", axis=1
    )
    
    # Color by miss rate
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=entity_stats_pd["proposer_entity_clean"],
        x=entity_stats_pd["miss_rate"],
        orientation="h",
        marker_color=entity_stats_pd["miss_rate"],
        marker_colorscale="YlOrRd",
        text=entity_stats_pd["label"],
        textposition="outside",
        hovertemplate="<b>%{y}</b><br>Miss rate: %{x:.2f}%<extra></extra>",
    ))
    fig.update_layout(
        margin=dict(l=150, r=100, t=30, b=60),
        xaxis=dict(title="Miss rate (%)", range=[0, max(entity_stats_pd["miss_rate"]) * 1.3]),
        yaxis=dict(title=""),
        height=max(300, len(entity_stats_pd) * 25 + 100),
    )
    fig.show(config={"responsive": True})

## Missed slots by time of day

Are there patterns in when slots are missed? Spikes could indicate coordinated issues or network events.

In [ ]:
if len(df_missed) > 0:
    # Extract hour from slot time
    df_missed_hourly = df_missed.with_columns(
        pl.col("slot_start_date_time").dt.hour().alias("hour")
    )
    
    hourly_misses = (
        df_missed_hourly.group_by("hour")
        .agg(pl.len().alias("missed_count"))
    )
    
    # Fill in missing hours with 0
    all_hours = pl.DataFrame({"hour": range(24)})
    hourly_misses = (
        all_hours.join(hourly_misses, on="hour", how="left")
        .fill_null(0)
        .sort("hour")
    )
    
    # Convert to pandas for plotting
    hourly_misses_pd = hourly_misses.to_pandas()
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=hourly_misses_pd["hour"],
        y=hourly_misses_pd["missed_count"],
        marker_color="#e74c3c",
    ))
    fig.update_layout(
        margin=dict(l=60, r=30, t=30, b=60),
        xaxis=dict(title="Hour (UTC)", tickmode="linear", dtick=2),
        yaxis=dict(title="Missed slots"),
        height=350,
    )
    fig.show(config={"responsive": True})

## Missed slots timeline

When did each missed slot occur? Clusters may indicate network-wide issues.

In [ ]:
if len(df_missed) > 0:
    # Convert to pandas for plotting (plotly needs pandas for datetime handling)
    df_plot = df_missed.select(["slot", "slot_start_date_time", "proposer_entity"]).to_pandas()
    df_plot["time"] = pd.to_datetime(df_plot["slot_start_date_time"])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_plot["time"],
        y=[1] * len(df_plot),  # All at same y level
        mode="markers",
        marker=dict(size=10, color="#e74c3c", symbol="x"),
        customdata=np.column_stack([df_plot["slot"], df_plot["proposer_entity"]]),
        hovertemplate="<b>Slot %{customdata[0]}</b><br>Time: %{x}<br>Entity: %{customdata[1]}<extra></extra>",
    ))
    fig.update_layout(
        margin=dict(l=60, r=30, t=30, b=60),
        xaxis=dict(title="Time (UTC)", tickformat="%H:%M"),
        yaxis=dict(visible=False),
        height=200,
    )
    fig.show(config={"responsive": True})

## All missed slots

Complete list of missed slots with proposer details.

In [ ]:
if len(df_missed) > 0:
    df_table = (
        df_missed.select(["slot", "slot_start_date_time", "proposer_entity"])
        .with_columns(
            pl.col("proposer_entity").fill_null("unknown").replace("", "unknown")
        )
        .with_columns(
            pl.col("slot_start_date_time").dt.strftime("%H:%M:%S").alias("time")
        )
        .sort("slot")
    )
    
    # Create Lab links
    df_table = df_table.with_columns(
        pl.col("slot").map_elements(
            lambda s: f'<a href="https://lab.ethpandaops.io/ethereum/slots/{s}" target="_blank">View</a>',
            return_dtype=pl.Utf8
        ).alias("lab_link")
    )
    
    # Build HTML table
    html = '''
    <style>
    .missed-table { border-collapse: collapse; width: 100%; font-family: monospace; font-size: 13px; }
    .missed-table th { background: #c0392b; color: white; padding: 8px 12px; text-align: left; position: sticky; top: 0; }
    .missed-table td { padding: 6px 12px; border-bottom: 1px solid #eee; }
    .missed-table tr:hover { background: #ffebee; }
    .missed-table a { color: #1976d2; text-decoration: none; }
    .missed-table a:hover { text-decoration: underline; }
    .table-container { max-height: 500px; overflow-y: auto; }
    </style>
    <div class="table-container">
    <table class="missed-table">
    <thead>
    <tr><th>Slot</th><th>Time (UTC)</th><th>Proposer entity</th><th>Lab</th></tr>
    </thead>
    <tbody>
    '''
    
    for row in df_table.iter_rows(named=True):
        html += f'''<tr>
            <td>{row["slot"]}</td>
            <td>{row["time"]}</td>
            <td>{row["proposer_entity"]}</td>
            <td>{row["lab_link"]}</td>
        </tr>'''
    
    html += '</tbody></table></div>'
    display(HTML(html))
    print(f"\nTotal missed slots: {len(df_table):,}")
else:
    print("No missed slots today.")